In [2]:
import cv2
import numpy as np
from keras.models import load_model
import time
import sys
# to suppress Pygame greeting
sys.stdout = open('nul', 'w')

import pygame
import tkinter as tk
from PIL import Image, ImageTk
import warnings


warnings.filterwarnings("ignore", category=DeprecationWarning)
# Initialize Tkinter
root = tk.Tk()
root.title("Drowsiness Detector")

# Loading the pre-trained drowsiness detection model
model = load_model('drowsiness_detector.h5')

# Loading Haar Cascade classifiers for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Initialize pygame mixer
pygame.mixer.init()

# Global variables
alert_active = False
prev_time = 0
monitoring = False
cap = None

# Function to predict eye state
def predict_eye_state(roi_gray):
    roi_gray = cv2.resize(roi_gray, (224, 224))  
    roi_gray = roi_gray / 255.0  
    roi_gray = np.expand_dims(roi_gray, axis=-1)  
    roi_gray = np.repeat(roi_gray, 3, axis=-1)  
    prediction = model.predict(np.expand_dims(roi_gray, axis=0))
    return prediction[0][0]  

# Function to play alert sound using pygame
def play_alert_sound():
    pygame.mixer.music.load('alert.mp3')
    pygame.mixer.music.play(loops=-1)  

# Function to stop alert sound
def stop_alert_sound():
    pygame.mixer.music.stop()

# Function to release camera and close windows
def release_camera_and_close_windows():
    global cap
    if cap is not None:
        cap.release()
    cv2.destroyAllWindows()
    if tk._default_root is not None:
        root.destroy()

# Function to start monitoring
def start_monitoring():
    global alert_active, prev_time, monitoring, cap

    cap = cv2.VideoCapture(0)

    monitoring = True
    while monitoring:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            roi_gray = gray[y:y + h, x:x + w]
            roi_color = frame[y:y + h, x:x + w]
            eyes = eye_cascade.detectMultiScale(roi_gray)

            for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
                roi_eye_gray = roi_gray[ey:ey + eh, ex:ex + ew]
                prediction = predict_eye_state(roi_eye_gray)

                if prediction < 0.5:
                    cv2.putText(frame, "Open", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    prev_time = time.time()
                    if alert_active:
                        stop_alert_sound()
                        alert_active = False
                else:
                    cv2.putText(frame, "Closed", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                    if not alert_active and time.time() - prev_time > 3:
                        play_alert_sound()  # Start alert if eyes closed for more than 3 seconds
                        alert_active = True

        # Convert the frame to RGB for displaying in tkinter
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Convert the frame to PIL format
        img = Image.fromarray(frame_rgb)
        imgtk = ImageTk.PhotoImage(image=img)

        # Check if the tkinter window is still active before updating the panel
        if not root.winfo_exists():
            break

        # Update the video panel with the new frame
        video_panel.imgtk = imgtk
        video_panel.configure(image=imgtk)
        video_panel.update()

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()

# Create GUI
# Create label for instructions
label = tk.Label(root, text="Press 'Start' to begin monitoring.")
label.pack()

# Create video panel to display webcam feed
video_panel = tk.Label(root)
video_panel.pack()

# Function to start monitoring when start button is clicked
def start_button_clicked():
    global cap
    start_button.config(state=tk.DISABLED)
    start_monitoring()

# Create start button
start_button = tk.Button(root, text="Start", command=start_button_clicked)
start_button.pack()

# Function to release camera and close windows when exit button is clicked
def exit_button_clicked():
    global cap
    stop_alert_sound()
    release_camera_and_close_windows()

# Create exit button
exit_button = tk.Button(root, text="Exit", command=exit_button_clicked)
exit_button.pack()

# Bind the release_camera_and_close_windows function to the window closure event
root.protocol("WM_DELETE_WINDOW", lambda: release_camera_and_close_windows())

# Start the tkinter main loop
root.mainloop()
